In [ ]:
!pip install paho-mqtt

In [ ]:
import paho.mqtt.client as mqtt
import time
from collections import deque
from sklearn import svm
import numpy as np

# 设置SVM模型来分类心跳数据
def train_svm():
    X_normal = np.random.randint(90, 121, (50, 1))
    y_normal = np.zeros(50)

    X_abnormal = np.concatenate([np.random.randint(60, 90, (25, 1)), np.random.randint(121, 150, (25, 1))])
    y_abnormal = np.ones(50)

    X = np.vstack((X_normal, X_abnormal))
    y = np.concatenate((y_normal, y_abnormal))

    model = svm.SVC(kernel='linear')
    model.fit(X, y)
    return model

model = train_svm()

broker1_address = "broker.emqx.io"
broker2_address = "test.mosquitto.org"
port = 1883
topic = "patient/+/heartbeat"  # 使用通配符来订阅多个主题

# 定义桥接模式: "in", "out", "both"
bridge_mode = "in"

class BridgeMQTTClient(mqtt.Client):
    def __init__(self, cname, **kwargs):
        super(BridgeMQTTClient, self).__init__(**kwargs)
        self.cname = cname
        self.on_connect = self.on_connect_callback
        self.on_message = self.on_message_callback
        self.processed_messages = deque(maxlen=100)

    def on_connect_callback(self, client, userdata, flags, rc):
        if rc == 0:
            print(f"{self.cname} connected to broker")
            if (self.cname == "bridge_c1" and bridge_mode in ["in", "both"]) or \
               (self.cname == "bridge_c2" and bridge_mode in ["out", "both"]):
                self.subscribe(topic)
        else:
            print(f"Connection failed with code {rc} for {self.cname}")

    def on_message_callback(self, client, userdata, msg):
        m_decode = str(msg.payload.decode("utf-8"))

        # 处理并发布消息
        print(f"Received message from {self.cname}: {m_decode}")
        heartbeat = int(m_decode)
        prediction = model.predict([[heartbeat]])
        if prediction == 0:
            print(f"Heartbeat from {msg.topic} is norma ")
        else:
            print("Heartbeat is abnormal")

        if self.cname == "bridge_c1" and bridge_mode in ["in", "both"]:
            bridge_client2.publish(msg.topic, m_decode)  # 保持原主题
            print(f"Forwarded message to broker2: {m_decode}")

        elif self.cname == "bridge_c2" and bridge_mode in ["out", "both"]:
            bridge_client1.publish(msg.topic, m_decode)  # 保持原主题
            print(f"Forwarded message to broker1: {m_decode}")

# 初始化第一个客户端，订阅并转发消息
bridge_client1 = BridgeMQTTClient(cname="bridge_c1")
bridge_client1.connect(broker1_address, port)
bridge_client1.loop_start()

# 初始化第二个客户端，接收转发的消息
bridge_client2 = BridgeMQTTClient(cname="bridge_c2")
bridge_client2.connect(broker2_address, port)
bridge_client2.loop_start()

# 保持主线程运行
try:
    while True:
        time.sleep(4)
except KeyboardInterrupt:
    pass
finally:
    print('now stop')
    bridge_client1.loop_stop()
    bridge_client2.loop_stop()
    bridge_client1.disconnect()
    bridge_client2.disconnect()
